In [98]:
%reload_kedro

[04/02/24 13:30:17] INFO     Resolved project path as:                                              ]8;id=391893;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=450854;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             /Users/tony/Documents/phd/courses/PoCS2/spiteful-allegory/work.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

[04/02/24 13:30:17] WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/fram ]8;id=302000;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=402390;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             ework/session/session.py:267: KedroDeprecationWarning: ConfigLoader                   
                             will be deprecated in Kedro 0.19. Please use the OmegaConfigLoader                    
                             instead. To consult the documentation for OmegaConfigLoader, see here:                
                             https://docs.kedro.org/en/stable/configuration/advanced_configuration.                
                             html#omegaconfigloader                                                                
                               warnings.warn(                                                                      
                                                                                                                   

                    WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/lazy_loade ]8;id=248998;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=28279;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             r/__init__.py:78: KedroDeprecationWarning: 'TextDataSet' has been                     
                             renamed to 'TextDataset', and the alias will be removed in                            
                             Kedro-Datasets 2.0.0                                                                  
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/lazy_loade ]8;id=148434;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=839199;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             r/__init__.py:78: KedroDeprecationWarning: 'CSVDataSet' has been                      
                             renamed to 'CSVDataset', and the alias will be removed in                             
                             Kedro-Datasets 2.0.0                                                                  
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    WARNING  /Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/lazy_loade ]8;id=645147;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=775772;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/warnings.py#109\109]8;;\
                             r/__init__.py:78: KedroDeprecationWarning: 'PickleDataSet' has been                   
                             renamed to 'PickleDataset', and the alias will be removed in                          
                             Kedro-Datasets 2.0.0                                                                  
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    INFO     Kedro project work                                                     ]8;id=778423;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=954283;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=939897;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=513253;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import requests
import xmltodict

In [106]:
ns = catalog.load('neurosynth_text')
nq = catalog.load('neuroquery_text')

def drop_duplicated_neuroquery(ns, nq):
    '''
    Find any articles that are in both databases.
    Drop them from neuroquery (less complete metadata)
    '''
    
    nq = nq[~nq['study_id'].isin(ns['study_id'])]
    return ns, nq

[04/02/24 13:34:30] INFO     Loading data from 'neurosynth_text' (CSVDataset)...                ]8;id=987273;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=339863;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py#502\502]8;;\

[04/02/24 13:34:31] INFO     Loading data from 'neuroquery_text' (CSVDataset)...                ]8;id=913641;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=87058;file:///Users/tony/anaconda3/envs/pocs/lib/python3.8/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [107]:
# get metadata for unique neuroquery studies

def get_pubmed_citations(nq):
    '''Query NIH API. Likely request-limited. Use caution.'''
    
    service_root= 'https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi'
    texts = nq['study_id']

    citations = []

    for text in texts:

        # compose query
        text = 'PMC' + text
        req = f'{service_root}?id={text}'

        # parse and hault out citation
        resp = requests.get(req)
        pub_meta = xmltodict.parse(resp.content)

        try:
            citation = pub_meta['OA']['records']['record']['@citation']
        except KeyError:
            citation = np.NaN

        citations.append(citation)

    nq_with_citations = pd.concat([
        nq, 
        pd.Series(citations, name='citation')], 
        axis=1)

    return nq_with_citations

def extract_dates_from_citations(df):

    pattern='[.|,|;]'

    expanded = df['citation'].str.split(pattern, regex=True, expand=True)
    expanded = expanded.astype('str')

    for c in expanded.columns:
            
        expanded[c] = pd.to_datetime(
            expanded[c].str.strip(), 
            format="%Y %b %d",
            errors='coerce')
        
    contracted = expanded.astype('str').sum(axis=1)
    contracted = pd.to_datetime(contracted.str.replace('NaT', ''))

    df['year'] = pd.DatetimeIndex(contracted).year

    return df



# nq_citations = get_pubmed_citations(nq)
# nq_cite = extract_dates_from_citations(nq)
nq_cite = pd.read_csv('../data/02_intermediate/nq_citations.csv')

nq_cite = drop_duplicated_neuroquery(ns, nq_cite)

In [108]:
def combine_texts(ns: pd.DataFrame, nq: pd.DataFrame):
    
    ns['source'] = 'neurosynth'
    nq['source'] = 'neuroquery'

    return pd.concat([ns, nq])

text = combine_texts(ns, nq_cite)
text

,id,study_id,contrast_id,abstract,authors,journal,year,title,source,citation
0,10022492-1,10022492,1,A fundamental characteristic of working memory...,"Callicott JH, Mattay VS, Bertolino A, Finn K, ...","Cerebral cortex (New York, N.Y. : 1991)",1999.0,Physiological characteristics of capacity cons...,neurosynth,NaN
1,10022494-1,10022494,1,Electrophysiological studies on monkeys have b...,"Toni I, Schluter ND, Josephs O, Friston K, Pas...","Cerebral cortex (New York, N.Y. : 1991)",1999.0,"Signal-, set- and movement-related activity in...",neurosynth,NaN
2,10022496-1,10022496,1,Most functional imaging studies of the auditor...,"Lockwood AH, Salvi RJ, Coad ML, Arnold SA, Wac...","Cerebral cortex (New York, N.Y. : 1991)",1999.0,The functional anatomy of the normal human aud...,neurosynth,NaN
3,10051677-1,10051677,1,Positron emission tomography studies were cond...,"Denton D, Shade R, Zamarippa F, Egan G, Blair-...",Proceedings of the National Academy of Science...,1999.0,Correlation of regional cerebral blood flow an...,neurosynth,NaN
4,10191322-1,10191322,1,The cortical organization of language in bilin...,"Chee MW, Tan EW, Thiel T",The Journal of neuroscience : the official jou...,1999.0,Mandarin and English single word processing st...,neurosynth,NaN
...,...,...,...,...,...,...,...,...,...,...
13450,28912749-1,28912749,1,The corpus callosum (CC) plays an important ro...,NaN,NaN,NaN,NaN,neuroquery,NaN
13451,28915853-1,28915853,1,"BACKGROUND: Human, hairy skin contains a subgr...",NaN,NaN,NaN,NaN,neuroquery,NaN
13452,28919871-1,28919871,1,The human brain has the capacity to integrate ...,NaN,NaN,NaN,NaN,neuroquery,NaN
13453,28924215-1,28924215,1,The human capacity to master multiple language...,NaN,NaN,NaN,NaN,neuroquery,NaN


In [111]:
text[text['source'] == 'neuroquery']

,id,study_id,contrast_id,abstract,authors,journal,year,title,source,citation
0,10202567-1,10202567,1,BACKGROUND: Patients with posttraumatic stress...,NaN,NaN,NaN,NaN,neuroquery,World J Nucl Med. 2023 May 1; 22(2):108-113
1,10439584-1,10439584,1,Impairment in semantic processing occurs early...,NaN,NaN,NaN,NaN,neuroquery,BMC Psychiatry. 2023 Aug 18; 23:604
2,10523407-1,10523407,1,1. Functional magnetic resonance imaging (fMRI...,NaN,NaN,NaN,NaN,neuroquery,Health Equity. 2023 Aug 30; 7(1):453-461
8,11273413-1,11273413,1,We used functional magnetic resonance imaging ...,NaN,NaN,NaN,NaN,neuroquery,NaN
10,11584070-1,11584070,1,Studies of delayed nonmatching-to-sample (DNMS...,NaN,NaN,NaN,NaN,neuroquery,NaN
...,...,...,...,...,...,...,...,...,...,...
13450,28912749-1,28912749,1,The corpus callosum (CC) plays an important ro...,NaN,NaN,NaN,NaN,neuroquery,NaN
13451,28915853-1,28915853,1,"BACKGROUND: Human, hairy skin contains a subgr...",NaN,NaN,NaN,NaN,neuroquery,NaN
13452,28919871-1,28919871,1,The human brain has the capacity to integrate ...,NaN,NaN,NaN,NaN,neuroquery,NaN
13453,28924215-1,28924215,1,The human capacity to master multiple language...,NaN,NaN,NaN,NaN,neuroquery,NaN
